In [9]:
# 1. load libraries
## Upload libraries
import sys
import pandas as pd
#upload HY funcitons
from Functions.calc_func import * 
from Functions.upload_functions import * 

# export file path:
expath = 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/csv_exports/'
tests = 'tests/'
ca_material_lib = 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/csv_exports/CA_materials_GFCFC_df/'




In [6]:
def attach_mat_class(emission_df):
    '''This function attaches the material classificaiton to a chosen enission df
    - updated emission classificaiton is material_lca_classification07.csv '''

    # # 1. import calssification
    path = 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/lists/'
    class_list = pd.read_csv(path + 'material_lca_classification07.csv')
    # arrange by country/commodity
    class_list.set_index(['country_province', 'commodity' ])

    # removes multi-index from LCA DF     left = on_res_commod_LCA.droplevel([1], axis = 1)
    left = emission_df
    #Creating a refernce index for both DF (country+commodity)
    left = left.reset_index()
    left['new_index'] = left['Unnamed: 0'] + ' ' + left['Unnamed: 1']
    left = left.set_index('new_index')
    left

    right = class_list
    right['new_index'] = right['country_province'] + ' ' + right['commodity']
    right = right.set_index('new_index')
    right

    #Joining DF
    material_df = left.join(right)
    return material_df

In [13]:
#Define lists
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON',
              'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
# Create an empty Canada DF for joining loop calculations
# 2. load files (material by residential)
prov_gfcfc_mat = pd.read_csv(ca_material_lib +  'CA-ONgfcfc_sub_materials.csv')
prov_gfcfc_mat.head(2)
#ca_res_mat_emission = pd.DataFrame(index = prov_gfcfc_mat.index)
ca_res_mat_emission = prov_gfcfc_mat[['Unnamed: 0','Unnamed: 1']]
ca_res_mat_emission


# 2. load province materials from librarty
for p in provinces:
    # 2. load files (material by residential)
    x = pd.read_csv(ca_material_lib +  p+'gfcfc_sub_materials.csv')
    #ca_material_lib +  p+'gfcfc_sub_materials.csv'
    print(p,'  file loaded')
    #3. isolate residential structures A1-3 A5 columns, 
    x_res = x.iloc[:, lambda x:[90,91]]
    #add prov res to canada province DF
    ca_res_mat_emission = ca_res_mat_emission.join(x_res, rsuffix=('_'+p))
ca_res_mat_emission

# 4. Attach material list funciton
x=ca_res_mat_emission
x= attach_mat_class(ca_res_mat_emission)
#x.head(2)


# 5. Group by material classificaiton
x = x.groupby(['lev1_name', 'lev2_name']).sum()
ca_res_mat_emission = x.drop(['index'], axis = 1)
ca_res_mat_emission = ca_res_mat_emission.drop(['lev1'], axis = 1)
ca_res_mat_emission

CA-NL   file loaded
CA-PE   file loaded
CA-NS   file loaded
CA-NB   file loaded
CA-QC   file loaded
CA-ON   file loaded
CA-MB   file loaded
CA-SK   file loaded
CA-AB   file loaded
CA-BC   file loaded
CA-YT   file loaded
CA-NT   file loaded
CA-NU   file loaded


C:\Users\Hatzav\AppData\Local\Temp\ipykernel_31612\1754324691.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  x = x.groupby(['lev1_name', 'lev2_name']).sum()


('CA-NL_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                              
ConstructionSector DirectEmissions                                                         2.357584e+05                   
                   Energy                                                                  7.746660e+06                   
Materials          Concrete                                                                2.399055e+07                   
                   EarthworkAndLandscaping_Materials                                       5.260987e+06                   
                   Energy                                                                  2.105112e+05                   
                   ExteriorImprovements                                                    4.581652e+06                   
                   Finishes                                                                9.991724e+06                   
                   MEP                                                                     1.956885e+07                   
                   Masonry                                                                 3.304561e+06                   
                   Metals                                                                  2.435139e+07                   
                   Openings                                                                3.643358e+06                   
                   OtherMaterials                                                          6.543948e+06                   
                   PlasticsAndComposites                                                   1.167226e+07                   
                   Raw_Materials                                                           2.969575e+06                   
                   Specialties                                                             8.545945e+02                   
                   ThermalAndMoistureProtection                                            2.984351e+06                   
                   Wood                                                                    6.052669e+07                   
Other              Margins                                                                 1.611023e+07                   
                   Other                                                                   1.745346e+06                   
Services           OtherServices                                                           2.481949e+07                   
Transport          TransportService                                                        2.582682e+07                   
                   Vehicle                                                                 1.607549e+03                   
Water              Water                                                                   8.183756e+04                   

                                                      ('CA-NL_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                                     
ConstructionSector DirectEmissions                                                         2.045684e+07                          
                   Energy                                                                  0.000000e+00                          
Materials          Concrete                                                                0.000000e+00                          
                   EarthworkAndLandscaping_Materials                                       0.000000e+00                          
                   Energy                                                                  0.000000e+00                          
                   ExteriorImprovements                             

In [15]:
#export to csv
ca_res_mat_emission.to_csv(expath+tests+'ca_res_mat_emission_by_prov.csv')


## All province Top 10 commodities

In [16]:
# 2. load files (material by residential)
prov_gfcfc_mat = pd.read_csv(ca_material_lib +  'Dataca_res_struct_CO2e.csv')
prov_gfcfc_mat.head(2)

,Unnamed: 0,Unnamed: 1,"('CA-NL', 'kg CO2 eq (short)')","('CA-PE', 'kg CO2 eq (short)')","('CA-NS', 'kg CO2 eq (short)')","('CA-NB', 'kg CO2 eq (short)')","('CA-QC', 'kg CO2 eq (short)')","('CA-ON', 'kg CO2 eq (short)')","('CA-MB', 'kg CO2 eq (short)')","('CA-SK', 'kg CO2 eq (short)')","('CA-AB', 'kg CO2 eq (short)')","('CA-BC', 'kg CO2 eq (short)')","('CA-YT', 'kg CO2 eq (short)')","('CA-NT', 'kg CO2 eq (short)')","('CA-NU', 'kg CO2 eq (short)')"
0,CA-AB,Canola (including rapeseed),8134.061117,5298.022157,34404.682726,25269.332650,459137.311552,864186.365798,66285.288224,53732.170892,896373.435193,818596.691155,3851.319971,2764.209088,528.135772
1,CA-AB,Oilseeds (except canola),116.812583,70.373384,446.114754,301.852831,4988.365737,11134.824688,1170.190182,968.623186,16411.398352,9353.525525,69.292236,55.647309,7.662397


In [151]:
# Attach material list funciton
x= attach_mat_class(prov_gfcfc_mat)
x.head(2)
# 3.2 - Group by material and sort 5 top commodities
#isolate residentials
right = pd.DataFrame(x.sum(axis=1)).reset_index()
right
#x.groupby(['lev2_name']).sum(axis =0).nlargest(5)
left = x[['lev1_name', 'lev2_name']].reset_index()
#create the new df
mat_df= left.join(right[0])

mat_df = mat_df.rename(columns={0:'b'})
mat_df= mat_df.rename(columns={'new_index':'c'})
mat_df= mat_df.rename(columns={'lev2_name':'a'})
mat_df

# Define a custom function to get the 5 largest values in column 'b'
def top10_largest(group):
    return group.nlargest(10, 'b')[['b', 'c']]

# Apply the custom function to each group in column 'a' using groupby
mat_top5 = mat_df.groupby('a').apply(top10_largest)
mat_top5
# rename column names
mat_top5 = mat_top5.rename(columns={'b':'ca_res_material_KGCO2e', 'c':'commodity'})
mat_top5 = mat_top5.rename(index={'a':'emission_source'})
mat_top5 = mat_top5.droplevel(1)
mat_top5

C:\Users\Hatzav\AppData\Local\Temp\ipykernel_39168\1553780555.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  right = pd.DataFrame(x.sum(axis=1)).reset_index()


,ca_res_material_KGCO2e,commodity
a,,
Concrete,7.035771e+08,CA-ON Ready-mixed concrete
Concrete,4.697450e+08,CA-BC Ready-mixed concrete
Concrete,3.490853e+08,CA-QC Ready-mixed concrete
Concrete,3.484564e+08,"CN Cement, lime and plaster"
Concrete,3.037294e+08,CA-AB Ready-mixed concrete
...,...,...
Wood,1.691627e+08,"CA-BC Wood products, n.e.c."
Wood,1.648556e+08,"CA-ON Wood products, n.e.c."
Wood,1.506805e+08,CA-ON Softwood lumber


In [145]:
mat_df.groupby('a').a.count()

a
Concrete                         137
DirectEmissions                  358
EarthworkAndLandscaping          160
Energy                          2002
ExteriorImprovements              52
Finishes                         290
MEP                              785
Margins                          299
Masonry                          173
Metals                          1484
Openings                          39
Other                           4524
OtherMaterials                   940
OtherServices                   2594
PlasticsAndComposites            290
Specialties                       13
ThermalAndMoistureProtection      39
TransportService                 890
Unprocessed                      215
Vehicle                          361
Water                            124
Wood                             427
Name: a, dtype: int64

In [152]:
mat_top5.to_csv(expath +tests+'CA_res_material_top10_commodity_emissions.csv')
mat_top5

,ca_res_material_KGCO2e,commodity
a,,
Concrete,7.035771e+08,CA-ON Ready-mixed concrete
Concrete,4.697450e+08,CA-BC Ready-mixed concrete
Concrete,3.490853e+08,CA-QC Ready-mixed concrete
Concrete,3.484564e+08,"CN Cement, lime and plaster"
Concrete,3.037294e+08,CA-AB Ready-mixed concrete
...,...,...
Wood,1.691627e+08,"CA-BC Wood products, n.e.c."
Wood,1.648556e+08,"CA-ON Wood products, n.e.c."
Wood,1.506805e+08,CA-ON Softwood lumber


In [5]:
x

index  \
lev1_name          lev2_name                                
ConstructionSector DirectEmissions                1896722   
                   Energy                        21874445   
Materials          Concrete                       1228793   
                   EarthworkAndLandscaping        1697535   
                   ExteriorImprovements            161837   
                   Finishes                       2122494   
                   MEP                            5285089   
                   Masonry                        1735207   
                   Metals                        13024135   
                   Openings                        121888   
                   OtherMaterials                 9327580   
                   PlasticsAndComposites          2105747   
                   Specialties                      41184   
                   ThermalAndMoistureProtection    122096   
                   Unprocessed                    1456920   
                   Wood                           3314915   
Other              Margins                         972972   
                   Other                         40085434   
Services           OtherServices                 14551028   
Transport          TransportService               7271290   
                   Vehicle                        1560032   
Water              Water                          1189767   

                                                 ('CA-NL_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')  \
lev1_name          lev2_name                                                                                         
ConstructionSector DirectEmissions                                                    2.357584e+05                   
                   Energy                                                             7.746660e+06                   
Materials          Concrete                                                           2.399055e+07                   
                   EarthworkAndLandscaping                                            5.260987e+06                   
                   ExteriorImprovements                                               4.581652e+06                   
                   Finishes                                                           9.991724e+06                   
                   MEP                                                                1.956885e+07                   
                   Masonry                                                            3.304561e+06                   
                   Metals                                                             2.435139e+07                   
                   Openings                                                           3.643358e+06                   
                   OtherMaterials                                                     6.543948e+06                   
                   PlasticsAndComposites                                              1.167226e+07                   
                   Specialties                                                        8.545945e+02                   
                   ThermalAndMoistureProtection                                       2.984351e+06                   
                   Unprocessed                                                        3.180086e+06                   
                   Wood                                                               6.052669e+07                   
Other              Margins                                                            1.611023e+07                   
                   Other                                                              1.745346e+06                   
Services           OtherServices                                                      2.481949e+07                   
Transport          TransportService                                                   2.582682e+07           

## Test

In [6]:
#Define lists
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON',
              'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
# Create an empty Canada DF for joining loop calculations
# 2. load files (material by residential)
prov_gfcfc_mat = pd.read_csv(ca_material_lib +  'CA-ONgfcfc_sub_materials.csv')
prov_gfcfc_mat.head(2)
#ca_res_mat_emission = pd.DataFrame(index = prov_gfcfc_mat.index)
ca_res_mat_emission = prov_gfcfc_mat[['Unnamed: 0','Unnamed: 1']]
ca_res_mat_emission


# 2. load province materials from librarty
for p in provinces:
    # 2. load files (material by residential)
    x = pd.read_csv(ca_material_lib +  p+'gfcfc_sub_materials.csv')
    #ca_material_lib +  p+'gfcfc_sub_materials.csv'
    print(p,'  file loaded')
    #3. isolate residential structures A1-3 A5 columns, 
    x_res = x.iloc[:, lambda x:[90,91]]
    #add prov res to canada province DF
    ca_res_mat_emission = ca_res_mat_emission.join(x_res, rsuffix=('_'+p))
ca_res_mat_emission

# 4. Attach material list funciton
x=ca_res_mat_emission
x= attach_mat_class(ca_res_mat_emission)
#x.head(2)

CA-NL   file loaded
CA-PE   file loaded
CA-NS   file loaded
CA-NB   file loaded
CA-QC   file loaded
CA-ON   file loaded
CA-MB   file loaded
CA-SK   file loaded
CA-AB   file loaded
CA-BC   file loaded
CA-YT   file loaded
CA-NT   file loaded
CA-NU   file loaded


In [19]:
energy_by_prov = x.loc[x.lev2_name == 'Energy' ]
energy_by_prov.to_csv(expath +tests+'enery_by_prov.csv')
energy_by_prov

,index,Unnamed: 0,Unnamed: 1,"('CA-NL_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-NL_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-PE_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-PE_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NS_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-NS_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NB_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')",...,"('CA-NT_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Residential structures_A1-A4', 'kg CO2 eq (short)')","('CA-NU_gfcfc_Residential structures_A5_finl_dmnd', 'kg CO2 eq (short)')",country_province,commodity,lev1,lev1_name,lev2,lev2_name,LCA
new_index,,,,,,,,,,,,,,,,,,,,,
CA-AB Natural gas,30,CA-AB,Natural gas,5504.481994,0.0,3872.077577,0.0,1.744787e+04,0.0,21174.868933,...,0.0,1211.969002,0.0,CA-AB,Natural gas,1,ConstructionSector,01 01,Energy,A1-A3
CA-AB Electricity,49,CA-AB,Electricity,1826.440689,0.0,3772.967733,0.0,3.933116e+03,0.0,5461.027823,...,0.0,1678.965101,0.0,CA-AB,Electricity,1,ConstructionSector,01 01,Energy,A1-A3
CA-AB Motor gasoline,145,CA-AB,Motor gasoline,1494.553617,0.0,401.132546,0.0,3.914579e+03,0.0,2310.024970,...,0.0,9213.543869,0.0,CA-AB,Motor gasoline,1,ConstructionSector,01 01,Energy,A1-A3
CA-AB Diesel and biodiesel fuels,146,CA-AB,Diesel and biodiesel fuels,2154.714861,0.0,645.015397,0.0,2.277620e+06,0.0,1454.344109,...,0.0,17247.632738,0.0,CA-AB,Diesel and biodiesel fuels,1,ConstructionSector,01 01,Energy,A1-A3
CA-AB Light fuel oils,147,CA-AB,Light fuel oils,8515.281232,0.0,5449.001818,0.0,3.084624e+06,0.0,1620.307421,...,0.0,1181.841409,0.0,CA-AB,Light fuel oils,1,ConstructionSector,01 01,Energy,A1-A3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WM Blast Furnace Gas,16138,WM,Blast Furnace Gas,0.066972,0.0,0.047724,0.0,2.107594e-01,0.0,0.146858,...,0.0,0.008883,0.0,WM,Blast Furnace Gas,1,ConstructionSector,01 01,Energy,0
WM Oxygen Steel Furnace Gas,16139,WM,Oxygen Steel Furnace Gas,0.030006,0.0,0.021045,0.0,9.303698e-02,0.0,0.066167,...,0.0,0.003911,0.0,WM,Oxygen Steel Furnace Gas,1,ConstructionSector,01 01,Energy,0
WM Gas Works Gas,16140,WM,Gas Works Gas,0.013710,0.0,0.009069,0.0,4.025555e-02,0.0,0.030833,...,0.0,0.001675,0.0,WM,Gas Works Gas,1,ConstructionSector,01 01,Energy,0
